# Techno-economic analysis

TEA objects can perform cashflow analysis on a System object and arguments for cashflow analysis. These include arguments such as operating days, lang factor, and income tax, as well as arguments for taking into account startup operation, construction schedule, and capital cost financing.

### Inhereting from TEA objects

Depending on the rigour and flexibility of the techno-economic analysis, different parameters may be needed to calculate total depreciable capital (TDC), fixed capital investment (FCI), and fixed operating cost (FOC). For this reason, the TEA object is left as an *abstract class* with *abstract methods* `_TDC`, `_FCI`, and `_FOC`. Here is an example TEA subclass from the lipidcane biorefinery design for the co-production of ethanol and biodiesel:

In [1]:
import biosteam as bst

class LipidcaneTEA(bst.TEA):
    """Create a TEA_lipidcane object for techno-economic analysis of a biorefinery [1]
    
    Parameters
        
        system: [System] Should contain feed and product streams.

        IRR: [float]  Internal rate of return (fraction).

        duration: tuple[int, int] Start and end year of venture (e.g. (2018, 2038)).

        depreciation: [str] 'MACRS' + number of years (e.g. 'MACRS7').

        operating_days: [float] Number of operating days per year.

        income_tax: [float] Combined federal and state income tax rate (fraction).

        lang_factor: [float] Lang factor for getting fixed capital investment from
        total purchase cost. If no lang factor, estimate capital investment
        using bare module factors.

        startup_schedule: tuple[float] Startup investment fractions per year 
        (e.g. (0.5, 0.5) for 50% capital investment in the first year and 50%
        investment in the second).

        WC_over_FCI: [float] Working capital as a fraction of fixed capital investment.

        labor_cost: [float] Total labor cost (USD/yr).

        fringe_benefits: [float] Cost of fringe benefits as a fraction of labor cost.

        property_tax: [float] Fee as a fraction of fixed capital investment.

        property_insurance: [float] Fee as a fraction of fixed capital investment.    

        supplies: [float] Yearly fee as a fraction of labor cost.

        maintenance: [float] Yearly fee as a fraction of fixed capital investment.

        administration: [float] Yearly fee as a fraction of fixed capital investment.

    References
    
        [1] Huang, H., Long, S., & Singh, V. (2016). Techno-economic analysis of biodiesel
        and ethanol co-production from lipid-producing sugarcane. Biofuels, Bioproducts
        and Biorefining, 10(3), 299–315. https://doi.org/10.1002/bbb.1640
    
    """
    
    def __init__(self, system, IRR, duration, depreciation, income_tax,
                 operating_days, lang_factor, construction_schedule, WC_over_FCI,
                 labor_cost, fringe_benefits, property_tax,
                 property_insurance, supplies, maintenance, administration):
        # Huang et. al. does not take into account financing or startup
        # so these parameters are 0 by default
        super().__init__(system, IRR, duration, depreciation, income_tax,
                         operating_days, lang_factor, construction_schedule,
                         startup_months=0, startup_FOCfrac=0, startup_VOCfrac=0,
                         startup_salesfrac=0, finance_interest=0, finance_years=0, 
                         finance_fraction=0, WC_over_FCI=WC_over_FCI)
        self.labor_cost = labor_cost
        self.fringe_benefits = fringe_benefits
        self.property_tax = property_tax
        self.property_insurance = property_insurance
        self.supplies= supplies
        self.maintenance = maintenance
        self.administration = administration
        
    # The abstract _TDC method should take direct permanent investment
    # and return the total depreciable capital. Huang et. al. assume 
    # these values are equal
    def _TDC(self, DPI):
        return DPI
    
    # The abstract _FCI method should take total depreciable capital
    # and return the fixed capital investment. Again, Huang et. al.  
    # assume these values are equal.
    def _FCI(self, TDC):
        return TDC
    
    # The abstract _FOC method should take fixed capital investment
    # and return the fixed operating cost.
    def _FOC(self, FCI):
        return (FCI*(self.property_tax + self.property_insurance
                     + self.maintenance + self.administration)
                + self.labor_cost*(1+self.fringe_benefits+self.supplies))

### Perform cash flow analysis

Create a TEA object from a system:

In [2]:
from biorefineries import lipidcane as lc

lipidcane_tea = LipidcaneTEA(system=lc.lipidcane_sys,
                             IRR=0.15,
                             duration=(2018, 2038),
                             depreciation='MACRS7',
                             income_tax=0.35,
                             operating_days=200,
                             lang_factor=3,
                             construction_schedule=(0.4, 0.6),
                             WC_over_FCI=0.05,
                             labor_cost=2.5e6,
                             fringe_benefits=0.4,
                             property_tax=0.001,
                             property_insurance=0.005,
                             supplies=0.20,
                             maintenance=0.01,
                             administration=0.005)

lipidcane_tea.show() # Print TEA summary at current options

LipidcaneTEA: lipidcane_sys
 NPV: 5.84e+07 USD at 15.0% IRR
 ROI: 0.177 1/yr
 PBP: 5.37 yr


Retrieve complete cashflow analysis as a DataFrame object:

In [3]:
lipidcane_tea.get_cashflow()

,Depreciable capital,Fixed capital investment,Working capital,Depreciation,Loan,...,Net earnings,Cash flow,Discount factor,Net present value (NPV),Cumulative NPV
2016,8.37e+07,8.37e+07,0,0,0,...,0,-8.37e+07,1.15,-9.62e+07,-9.62e+07
2017,1.26e+08,1.26e+08,1.05e+07,0,0,...,0,-1.36e+08,1,-1.36e+08,-2.32e+08
2018,0,0,0,2.99e+07,0,...,1.96e+07,4.95e+07,0.87,4.3e+07,-1.89e+08
2019,0,0,0,5.12e+07,0,...,5.69e+06,5.69e+07,0.756,4.3e+07,-1.46e+08
2020,0,0,0,3.66e+07,0,...,1.52e+07,5.18e+07,0.658,3.41e+07,-1.12e+08
2021,0,0,0,2.61e+07,0,...,2.2e+07,4.81e+07,0.572,2.75e+07,-8.46e+07
2022,0,0,0,1.87e+07,0,...,2.68e+07,4.55e+07,0.497,2.26e+07,-6.2e+07
2023,0,0,0,1.87e+07,0,...,2.69e+07,4.55e+07,0.432,1.97e+07,-4.23e+07
2024,0,0,0,1.87e+07,0,...,2.68e+07,4.55e+07,0.376,1.71e+07,-2.52e+07
2025,0,0,0,9.33e+06,0,...,3.29e+07,4.23e+07,0.327,1.38e+07,-1.13e+07


### Find production cost, price, and IRR

Find production cost:

In [4]:
products = [bst.find('ethanol'), bst.find('biodiesel')]
lipidcane_tea.production_cost(products)# USD/yr

array([23883703.917, 30848483.181])

Solve for the price of a stream at the break even point:

In [5]:
feed = bst.find('lipid_cane')
lipidcane_tea.solve_price(feed) # USD/kg

0.04351765779883601

Solve for the IRR at the break even point:

In [6]:
lipidcane_tea.IRR = lipidcane_tea.solve_IRR()
lipidcane_tea.show()

LipidcaneTEA: lipidcane_sys
 NPV: -0.00106 USD at 19.5% IRR
 ROI: 0.177 1/yr
 PBP: 5.37 yr


Note that the cashflow table is created up to date:

In [7]:
lipidcane_tea.get_cashflow()

,Depreciable capital,Fixed capital investment,Working capital,Depreciation,Loan,...,Net earnings,Cash flow,Discount factor,Net present value (NPV),Cumulative NPV
2016,8.37e+07,8.37e+07,0,0,0,...,0,-8.37e+07,1.19,-1e+08,-1e+08
2017,1.26e+08,1.26e+08,1.05e+07,0,0,...,0,-1.36e+08,1,-1.36e+08,-2.36e+08
2018,0,0,0,2.99e+07,0,...,1.96e+07,4.95e+07,0.837,4.14e+07,-1.95e+08
2019,0,0,0,5.12e+07,0,...,5.69e+06,5.69e+07,0.701,3.99e+07,-1.55e+08
2020,0,0,0,3.66e+07,0,...,1.52e+07,5.18e+07,0.587,3.04e+07,-1.24e+08
2021,0,0,0,2.61e+07,0,...,2.2e+07,4.81e+07,0.491,2.36e+07,-1.01e+08
2022,0,0,0,1.87e+07,0,...,2.68e+07,4.55e+07,0.411,1.87e+07,-8.19e+07
2023,0,0,0,1.87e+07,0,...,2.69e+07,4.55e+07,0.344,1.57e+07,-6.62e+07
2024,0,0,0,1.87e+07,0,...,2.68e+07,4.55e+07,0.288,1.31e+07,-5.31e+07
2025,0,0,0,9.33e+06,0,...,3.29e+07,4.23e+07,0.241,1.02e+07,-4.29e+07


### System report

Save stream tables, utility requirements, itemized costs, TEA results, and a cash flow table:

In [8]:
# Try this on your computer and open excel
lc.lipidcane_sys.save_report('Lipidcane report.xlsx')